# Un problema de árboles de decisión y pipelines

En este taller estudiarás conceptos mostrados en los tutoriales de "Solución de problemas con pipelines" para el preprocesamiento y búsqueda de hiperparámetros, utilizando el conjunto de datos correspondiente al desempeño y desgaste de empleados. Particularmente, realizarás los siguientes procesos:

1. Cargar un conjunto de datos.
2. Definir los pasos del preprocesamiento.
3. Realizar la búsqueda de hiperparámetros con un pipeline.
4. Evaluar el mejor modelo resultante.

Entonces, dadas algunas características en áreas como la educación, trabajos previos, salario, entre otras, queremos clasificar a un empleado como agotado (con bajo desempeño) o no. Antes de iniciar, vamos a importar las librerías necesarias:

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

## 1. Carga de datos

Con las librerías importadas, realizaremos la carga del conjunto de datos:

### Ejercicio 1.1.

Utiliza Pandas para importar el archivo que contiene el conjunto de datos de desgaste de empleados.

* La ruta del archivo .csv es: `./data/EmployeeAttrition.csv`, y ya se encuentra en el entorno de Coursera, solo debes importarlo.
* La variable resultante debe tener el nombre `data_raw`, que representa el conjunto de datos sin modificar.

In [3]:
ruta = './data/Employee-Attrition.csv'
# your code here
data_raw = pd.read_csv(ruta, sep=',')

In [4]:
#---------- Celda de Pruebas ----------
# El resultado existe
# El resultado es un DataFrame
# El resultado tiene las dimensiones correctas
#--------------------------------------

# Se verifica que la variable exista
assert data_raw is not None, "Asegúrate de definir la variable \'data_raw\' igualándola a una función de Pandas para leer el archivo."

# Se verifica que sea un DataFrame
assert isinstance(data_raw, pd.DataFrame), "El resultado debe ser un DataFrame."

# Se evalúan las dimensiones de la variable
assert data_raw.shape == (1470,32), "¿Verificaste que la ruta del archivo CSV y el nombre de la variable son correctos?"
print("¡Los datos tienen las dimensiones correctas!")

¡Los datos tienen las dimensiones correctas!


## 2. Definición de pasos del preprocesamiento

Primero vamos a definir la variable `data` para almacenar un conjunto de datos modificado:

In [5]:
data = data_raw.copy()

Eliminaremos la variable `EmployeeNumber`, correspondiente al identificador del empleado:

In [6]:
data = data.drop('EmployeeNumber', axis=1)

A continuación verificaremos si hay filas duplicadas:

In [7]:
data.duplicated().sum()

0

Además, verificaremos si hay valores faltantes:

In [8]:
data.isna().sum()

Age                         20
Attrition                    0
BusinessTravel              41
DailyRate                   18
Department                  37
DistanceFromHome             0
Education                   39
EducationField              53
EnvironmentSatisfaction     19
Gender                      58
HourlyRate                  31
JobInvolvement               6
JobLevel                    47
JobRole                     53
JobSatisfaction             33
MaritalStatus               46
MonthlyIncome               36
MonthlyRate                 59
NumCompaniesWorked          60
OverTime                    55
PercentSalaryHike           23
PerformanceRating           56
RelationshipSatisfaction     3
StockOptionLevel             5
TotalWorkingYears           11
TrainingTimesLastYear       63
WorkLifeBalance              6
YearsAtCompany              11
YearsInCurrentRole          71
YearsSinceLastPromotion     70
YearsWithCurrManager        32
dtype: int64

### Ejercicio 2.1.

Primero vamos a dividir el conjunto de datos en entrenamiento y pruebas. Usando el 80% de los datos para entrenar el modelo y el 20% restante para probarlo, utiliza `scikit-learn` para separar el conjunto de datos en dos.

* Guarda tu respuesta en dos variables: `train` y `test`. (**Ejemplo: `train, test = <<Función>>`**)
* Utiliza el parámetro `random_state=0`. Esto hará que la partición sea siempre la misma.
* Encontrarás la línea `train.head()`. Esta línea se usa para que puedas visualizar el resultado del conjunto de entrenamiento. Déjala al final de la celda y no la modifiques.

In [9]:
# your code here
train, test = train_test_split(data, test_size=0.2, random_state=0)
train.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
1374,58.0,0,Travel_Rarely,605.0,Sales,21.0,3.0,Life Sciences,4.0,Female,...,3.0,3.0,1.0,29.0,2.0,2.0,1.0,0.0,0.0,0.0
1092,45.0,0,Travel_Rarely,950.0,Research & Development,28.0,3.0,Technical Degree,4.0,Male,...,4.0,4.0,1.0,8.0,3.0,3.0,5.0,4.0,0.0,3.0
768,40.0,0,Travel_Rarely,300.0,Sales,26.0,3.0,NaN,3.0,Male,...,3.0,2.0,1.0,8.0,3.0,2.0,7.0,7.0,7.0,5.0
569,36.0,0,Non-Travel,1434.0,Sales,8.0,4.0,Life Sciences,1.0,Male,...,3.0,2.0,0.0,10.0,1.0,3.0,10.0,7.0,0.0,9.0
911,25.0,1,Travel_Frequently,599.0,Sales,24.0,1.0,Life Sciences,3.0,Male,...,3.0,4.0,0.0,1.0,4.0,3.0,1.0,0.0,1.0,0.0


In [10]:
#---------- Celda de Pruebas ----------
# Las variables "train" y "test" existen
# Las variables "train" y "test" son un DataFrame
# Las variables tienen las dimensiones correctas
#--------------------------------------

# Se verifica que "train" y "test" están definidas
assert train is not None, "Asegúrate de definir la variable \'train\' con el nombre correcto."
assert test is not None, "Asegúrate de definir la variable \'test\' con el nombre correcto."

# Se verifica que "train" y "test" sean un DataFrame
assert isinstance(train, pd.DataFrame), "La variable \'train\' debe ser un DataFrame."
assert isinstance(test, pd.DataFrame), "La variable \'test\' debe ser un DataFrame."

# Se evalúan las dimensiones de las variables
assert train.shape == (1176,31), "Verifica que estés utilizando el 80% de los datos para el conjunto de entrenamiento."
assert test.shape == (294,31), "Verifica que estés utilizando el 20% de los datos para el conjunto de pruebas."
print("¡Los conjuntos de entrenamiento y pruebas tienen las dimensiones correctas!")

¡Los conjuntos de entrenamiento y pruebas tienen las dimensiones correctas!


### Ejercicio 2.2.

Ahora debes aislar la variable objetivo, `Attrition`, de las variables independientes. Utiliza Pandas para crear dos variables, `x_train` y `y_train`, que almacenarán las variables independientes y la variable objetivo, respectivamente.

* Crea una variable con nombre `x_train` y asígnale la operación necesaria para almacenar solo las variables independientes del conjunto de entrenamiento. (**Ejemplo: `x_train = train.<<Función>>`**)
* Crea una variable con nombre `y_train` y asígnale la operación necesaria para almacenar la variable objetivo del conjunto de entrenamiento. (**Ejemplo: `y_train = <<Consulta>>`**)

In [11]:
# Tu respuesta deben ser dos líneas consecutivas:
#    x_train = train.<<Función>>
#    y_train = <<Consulta>>
# your code here
x_train = train.drop(['Attrition'],axis=1)
y_train = train['Attrition']

In [12]:
#---------- Celda de Pruebas ----------
# Las variables "x_train" y "y_train" existen
# La variable "x_train" es un DataFrame
# La variable "y_train" es una Serie de Pandas
# Las variables tienen las dimensiones correctas
#--------------------------------------

# Se verifica que las variables están definidas
assert x_train is not None, "Asegúrate de definir la variable \'x_train\' correctamente."
assert y_train is not None, "Asegúrate de definir la variable \'y_train\' correctamente."

# Se verifica que "x_train" sea un DataFrame
assert isinstance(x_train, pd.DataFrame), "El resultado debe ser un DataFrame."

# Se verifica que "y_train" sea una Serie
assert isinstance(y_train, pd.Series), "El resultado debe ser una Serie de Pandas."

# Se evalúan las dimensiones de las variables
assert x_train.shape == (1176,30), "\'x_train\' debe tener el mismo número de filas pero una columna menos que \'train\'."
assert y_train.shape == (1176,), "\'y_train\' solamente contiene una columna."
print("¡Los conjuntos \'x_train\' y \'y_train\' tienen las dimensiones correctas!")

¡Los conjuntos 'x_train' y 'y_train' tienen las dimensiones correctas!


### Ejercicio 2.3.

#### Ejercicio 2.3.1.

A continuación, vas a definir las técnicas de imputación para los valores faltantes. Utiliza `scikit-learn` para definir dos imputadores con las siguientes características:

* Un imputador para las variables numéricas que utilice la **media** como estrategia. Asigna tu respuesta a una nueva variable con el nombre `num_imputer`. (**Ejemplo: `num_imputer = <<Expresión>>`**)
* Un imputador para las variables categóricas que utilice el **valor más frecuente** como estrategia. Asigna tu respuesta a una nueva variable con el nombre `cat_imputer`. (**Ejemplo: `cat_imputer = <<Expresión>>`**)

In [13]:
# Tu respuesta deben ser dos líneas consecutivas:
#    num_imputer = <<Expresión>>
#    cat_imputer = <<Expresión>>
# your code here
cat_imputer = SimpleImputer(strategy='most_frequent')
num_imputer = SimpleImputer(strategy='mean')

In [14]:
#---------- Celda de Pruebas ----------
# Las variables 'num_imputer' y 'cat_imputer' existen
# Las variables son objetos para imputación de la librería scikit-learn
#--------------------------------------

# Se verifica que la variable 'num_imputer' existe
assert num_imputer is not None, "Asegúrate de definir la variable \'num_imputer\' con una clase de \'scikit-learn\'."

# Se verifica que la variable 'cat_imputer' existe
assert cat_imputer is not None, "Asegúrate de definir la variable \'cat_imputer\' con una clase de \'scikit-learn\'."
print("¡Los objetos de imputación son correctos!")

¡Los objetos de imputación son correctos!


#### Ejercicio 2.3.2.

A continuación vas a definir un `ColumnTransformer()` para juntar los imputadores en un único paso. Vamos a identificar las variables numéricas y categóricas utilizando el atributo `dtype` del conjunto de entrenamiento y comparando con tipos de numpy (`np.dtype()`):

In [15]:
# Variables numéricas: su tipo es float64 o int64
numeric_features = [
    i for i, col in enumerate(x_train.columns) 
    if (x_train.dtypes[col]==np.dtype('int64'))
    or (x_train.dtypes[col]==np.dtype('float64'))
]
# Variables categóricas: su tipo es object
categorical_features = [
    i for i, col in enumerate(x_train.columns) 
    if (x_train.dtypes[col]==np.dtype('object'))
]

# Veremos las tuplas (índice, variable) para los dos tipos
print(f"Variables numéricas:\n{list(zip(numeric_features, x_train.columns.take(numeric_features)))}")
print(f"\nVariables categóricas:\n{list(zip(categorical_features, x_train.columns.take(categorical_features)))}")

Variables numéricas:
[(0, 'Age'), (2, 'DailyRate'), (4, 'DistanceFromHome'), (5, 'Education'), (7, 'EnvironmentSatisfaction'), (9, 'HourlyRate'), (10, 'JobInvolvement'), (11, 'JobLevel'), (13, 'JobSatisfaction'), (15, 'MonthlyIncome'), (16, 'MonthlyRate'), (17, 'NumCompaniesWorked'), (19, 'PercentSalaryHike'), (20, 'PerformanceRating'), (21, 'RelationshipSatisfaction'), (22, 'StockOptionLevel'), (23, 'TotalWorkingYears'), (24, 'TrainingTimesLastYear'), (25, 'WorkLifeBalance'), (26, 'YearsAtCompany'), (27, 'YearsInCurrentRole'), (28, 'YearsSinceLastPromotion'), (29, 'YearsWithCurrManager')]

Variables categóricas:
[(1, 'BusinessTravel'), (3, 'Department'), (6, 'EducationField'), (8, 'Gender'), (12, 'JobRole'), (14, 'MaritalStatus'), (18, 'OverTime')]


Completa el siguiente código para definir el parámetro `transformers` del objeto `imputer`:

* Utiliza las variables `numeric_features` y `categorical_features` para indicar qué objeto va a operar sobre qué variables.
* Utiliza el nombre `"num"` para las variables numéricas y el nombre `"cat"` para las variables categóricas.

In [16]:
imputer = ColumnTransformer(
    transformers=[
    # your code here
        ("num", num_imputer, numeric_features),
        ("cat", cat_imputer, categorical_features)
    ]
)

In [17]:
#---------- Celda de Pruebas ----------
# La variable 'imputer' existe
# La variable 'imputer' es un objeto de la clase ColumnTransformer
#--------------------------------------

# Se verifica que la variable 'imputer' existe
assert imputer is not None, "Asegúrate de definir la variable \'imputer\' correctamente."

# Se verifica que la variable 'imputer' sea un objeto de la clase ColumnTransformer
assert type(imputer) == ColumnTransformer, "La variable \'imputer\' debe ser un objeto de la clase ColumnTransformer."
print("¡La variable \'imputer\' está bien definida!")

¡La variable 'imputer' está bien definida!


### Ejercicio 2.4.

Como el modelo de árboles de decisión no requiere estandarización de los datos, solamente definiremos el paso de codificación OneHot para las variables categóricas con `scikit-learn`:

#### Ejercicio 2.4.1.

Inicializa un objeto de la clase necesaria para realizar la codificación OneHot. Almacena el objeto en una nueva variable con el nombre `onehot_encoder` (**Ejemplo: `onehot_encoder = <<Expresión>>`**)

In [18]:
# your code here
onehot_encoder = OneHotEncoder()

In [19]:
#---------- Celda de Pruebas ----------
# La variable 'onehot_encoder' existe
# La variable 'onehot_encoder' es un objeto de la clase correcta
#--------------------------------------

# Se verifica que la variable 'onehot_encoder' existe
assert onehot_encoder is not None, "Asegúrate de definir la variable \'onehot_encoder\' correctamente."
print("¡La variable \'onehot_encoder\' está bien definida!")

¡La variable 'onehot_encoder' está bien definida!


#### Ejercicio 2.4.2.

A continuación vas a definir un `ColumnTransformer()` para realizar la codificación OneHot únicamente sobre las variables categóricas. Como ha cambiado el orden de las variables, vamos a redefinir `numeric_features` y `categorical_features`:

In [20]:
# Variables numéricas: ahora son las primeras 23 posiciones
numeric_features = list(range(23))
# Variables categóricas: BusinessTravel, Department, EducationField, Gender, JobRole, MaritalStatus, OverTime
categorical_features = list(range(23,30))

Completa el siguiente código para definir el parámetro `transformers` del objeto `encoder`:

* Utiliza la variable `categorical_features` para indicar qué variables van a ser modificadas, y la variable `onehot_encoder` para definir la estrategia de codificación.
* Utiliza el nombre `"cat"` para definir la tupla correspondiente a las variables categóricas.
* Encontrarás la tupla `("num", 'passthrough', numeric_features)`, que indica que las variables numéricas no van a ser alteradas. No debes modificar esta expresión.

In [21]:
encoder = ColumnTransformer(
    transformers=[
        ("num", 'passthrough', numeric_features),
    # your code here
        ("cat", onehot_encoder, categorical_features)
    
    ]
)

In [22]:
#---------- Celda de Pruebas ----------
# La variable 'encoder' existe
# La variable 'encoder' es un objeto de la clase ColumnTransformer
#--------------------------------------

# Se verifica que la variable 'encoder' existe
assert encoder is not None, "Asegúrate de definir la variable \'encoder\' correctamente."

# Se verifica que la variable 'encoder' sea un objeto de la clase ColumnTransformer
assert type(encoder) == ColumnTransformer, "La variable \'encoder\' debe ser un objeto de la clase ColumnTransformer."
print("¡La variable \'encoder\' está bien definida!")

¡La variable 'encoder' está bien definida!


## 3. Búsqueda de hiperparámetros con un pipeline

Con el conjunto de entrenamiento preparado, definiremos un último paso antes de construir el pipeline: el objeto de la clase `DecisionTreeClassifier` para realizar el entrenamiento con un árbol de decisión. Ten en cuenta que usamos el parámetro `random_state=0` para que, después de definir una división, la reorganización de las variables restantes sea siempre la misma:

In [23]:
decision_tree = DecisionTreeClassifier(random_state=0)

Con el último paso definido, podemos agrupar todas las etapas en un arreglo que utilizarás para definir el pipeline:

In [24]:
steps = [
    ("imputer", imputer),
    ("encoder", encoder),
    ("model", decision_tree),
]

Finalmente, definiremos la variable `kfold` para realizar la validación cruzada con 5 subconjuntos de datos:

In [25]:
kfold = KFold(n_splits=5, shuffle=True, random_state=0)

### Ejercicio 3.1.

Ahora vas a crear el pipeline de procesamiento y entrenamiento para nuestro problema.

* Define una variable con el nombre `pipeline` y asígnale la expresión necesaria para definir un pipeline con los pasos almacenados en la variable `steps` (**Ejemplo: `pipeline = <<Expresión>>`**):

In [26]:
# your code here
pipeline = Pipeline(steps)

In [27]:
#---------- Celda de Pruebas ----------
# La variable 'pipeline' existe
# La variable 'pipeline' es un objeto de la clase Pipeline
# La variable 'pipeline' tiene pasos bien definidos
#--------------------------------------

# Se verifica que la variable 'pipeline' existe
assert pipeline is not None, "Asegúrate de definir la variable \'pipeline\' correctamente."

# Se verifica que la variable 'pipeline' sea un objeto de la clase Pipeline
assert type(pipeline) == Pipeline, "La variable \'pipeline\' debe ser un objeto de la clase Pipeline."
print("¡La variable \'pipeline\' está bien definida!")

¡La variable 'pipeline' está bien definida!


### Ejercicio 3.2.

El siguiente paso es definir el espacio de búsqueda de los hiperparámetros. En este caso, realizaremos una búsqueda incluyendo la estrategia de imputación para variables numéricas.

* Define una variable con el nombre `param_grid` y asígnale la expresión necesaria para crear un diccionario con cuatro tuplas (**Ejemplo: `param_grid = <<Expresión>>`**):
    * Llave `imputer__num__strategy` y valor `valores_imputer`.
    * Llave `model__criterion` y valor `valores_criterion`.
    * Llave `model__max_depth` y valor `valores_max_depth`.
    * Llave `model__min_samples_split` y valor `valores_min_samples_split`.

In [28]:
valores_imputer = ['mean', 'median']
valores_criterion = ['entropy', 'gini']
valores_max_depth = [2, 4, 6]
valores_min_samples_split = [2, 3, 5]
# your code here
param_grid = {
    # Paso de imputación
    'imputer__num__strategy': valores_imputer,
    # Paso de modelado
    'model__criterion': valores_criterion,
    'model__max_depth': valores_max_depth,
    'model__min_samples_split': valores_min_samples_split
}

In [29]:
#---------- Celda de Pruebas ----------
# La variable "param_grid" existe
# La variable "param_grid" es un diccionario
# La variable tiene la longitud correcta
#--------------------------------------

# Se verifica que "param_grid" está definida
assert param_grid is not None, "Asegúrate de definir la variable \'param_grid\' correctamente."

# Se verifica que "param_grid" sea un diccionario
assert isinstance(param_grid, dict), "El resultado debe ser un diccionario de Python, es decir, no necesitas ninguna función específica de ninguna librería para definir la variable."

# Se evalúa la longitud de la variable
assert len(param_grid) == 4, "\'param_grid\' debe contener cuatro tuplas."
print("¡\'param_grid\' está definida correctamente!")

¡'param_grid' está definida correctamente!


### Ejercicio 3.3.

Finalmente, el último paso antes de realizar la búsqueda de hiperparámetros es crear el objeto de tipo `GridSearchCV`. Utiliza las variables `pipeline`, `param_grid` y `kfold` para definirlo:

* Define una variable con el nombre `grid` y asígnale la función necesaria para crear un objeto de la clase `GridSearchCV`. (**Ejemplo: `grid = <<Función>>`**)
* Utiliza el parámetro `scoring='accuracy'` para que se seleccione el mejor modelo de acuerdo con los valores de exactitud.

In [30]:
# your code here
grid = GridSearchCV(pipeline, param_grid, cv=kfold,scoring='accuracy', n_jobs=-1)

In [31]:
#---------- Celda de Pruebas ----------
# La variable "grid" existe
# La variable "grid" es un objeto de la clase GridSearchCV
# La variable "grid" usa la exactitud como método de selección
#--------------------------------------

# Se verifica que "grid" está definida
assert grid is not None, "Asegúrate de definir la variable \'grid\' correctamente."

# Se verifica que "grid" es un objeto de la clase GridSearchCV
assert type(grid) == GridSearchCV, "La variable \'grid\' debe ser un objeto de la clase GridSearchCV."

# Se verifica que "grid" tiene un atributo "scoring" igual a "accuracy"
assert grid.scoring == 'accuracy', "La variable \'grid\' debe usar la exactitud como método de selección."
print("¡\'grid\' está definida correctamente!")

¡'grid' está definida correctamente!


### Ejercicio 3.4.

A continuación, realiza la búsqueda de hiperparámetros utilizando el conjunto de entrenamiento, compuesto por las variables `x_train` y `y_train`.

* Para este ejercicio no debes asignar tu resultado a ninguna variable. Es decir, solo debes ejecutar una función sobre la variable `grid`, utilizando las variables `x_train` y `y_train` como parámetros. (**Ejemplo: `grid.<<Función>>`**)

In [32]:
# your code here
grid.fit(x_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('imputer',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num',
                                                                         SimpleImputer(add_indicator=False,
                                                                                       copy=True,
                                                                                       fill_value=None,
                                                                                       missing_values=nan,
 

In [33]:
#---------- Celda de Pruebas ----------
# El atributo "best_params_" de la variable "grid" existe
# El atributo "best_estimator_" de la variable "grid" existe
#--------------------------------------

# El atributo "best_params_" está definido
assert grid.best_params_ is not None, "Asegúrate de ejecutar la función de entrenamiento para generar un diccionario con los mejores hiperparámetros."

# El atributo "best_estimator_" está definido
assert grid.best_estimator_ is not None, "Asegúrate de ejecutar la función de entrenamiento para generar un modelo de árboles de decisión con preprocesamiento."

# Se verifica que "grid" haya generado cuatro mejores hiperparámetros
assert len(grid.best_params_) == 4, "Al ejecutar una función usando el objeto \'grid\', se deben generar cuatro tuplas con los hiperparámetros."

# Se verifica que "grid" haya generado un pipeline
assert isinstance(grid.best_estimator_, Pipeline), "Al ejecutar una función usando el objeto \'grid\', se debe generar un pipeline."
print("¡Se ha realizado la búsqueda de hiperparámetros correctamente!")

¡Se ha realizado la búsqueda de hiperparámetros correctamente!


Ahora obtendremos los mejores valores de los hiperparámetros usando `grid.best_params_`:

In [34]:
print("Mejores parámetros: {}".format(grid.best_params_)) 

Mejores parámetros: {'imputer__num__strategy': 'mean', 'model__criterion': 'entropy', 'model__max_depth': 2, 'model__min_samples_split': 2}


Además, almacenaremos el mejor modelo utilizando `grid.best_estimator_`:

In [35]:
mejor_modelo = grid.best_estimator_

## 4. Evaluación del modelo

Por último, evaluarás el modelo entrenado utilizando el conjunto de pruebas.

### Ejercicio 4.1.

Separa las variables independientes y la variable objetivo en el conjunto de pruebas. Para ello, utiliza Pandas para crear dos variables, `x_test` y `y_test`, que almacenarán las variables independientes y la variable objetivo, respectivamente.

* Crea una variable con nombre `x_test` y asígnale la operación necesaria para almacenar solo las variables independientes del conjunto de pruebas. (**Ejemplo: `x_test = test.<<Función>>`**)
* Crea una variable con nombre `y_test` y asígnale la operación necesaria para almacenar la variable objetivo del conjunto de pruebas. (**Ejemplo: `y_test = <<Consulta>>`**)

In [36]:
# Tu respuesta deben ser dos líneas consecutivas:
#    x_test = test.<<Función>>
#    y_test = <<Consulta>>
# your code here
x_test = test.drop(['Attrition'],axis=1)
y_test = test['Attrition']

In [37]:
#---------- Celda de Pruebas ----------
# Las variables existen
# La variable "x_test" es un DataFrame
# La variable "y_test" es una Serie de Pandas
# Las variables tienen las dimensiones correctas
#--------------------------------------

# Se verifica que las variables están definidas
assert x_test is not None, "Asegúrate de definir la variable con el nombre correcto."
assert y_test is not None, "Asegúrate de definir la variable con el nombre correcto."

# Se verifica que "x_test" sea un DataFrame
assert isinstance(x_test, pd.DataFrame), "El resultado debe ser un DataFrame."

# Se verifica que "y_test" sea una Serie
assert isinstance(y_test, pd.Series), "El resultado debe ser una Serie de Pandas."

# Se evalúan las dimensiones de las variables
assert x_test.shape == (294,30), "\'x_test\' debe tener el mismo número de filas pero una columna menos que \'test\'."
assert y_test.shape == (294,), "\'y_test\' solamente contiene una columna."
print("¡Los conjuntos \'x_test\' y \'y_test\' tienen las dimensiones correctas!")

¡Los conjuntos 'x_test' y 'y_test' tienen las dimensiones correctas!


### Ejercicio 4.2.

Con el conjunto de pruebas preparado, realiza predicciones con el fin de compararlas con los valores reales almacenados en `y_test`.

* Utiliza la variable `mejor_modelo` para realizar las predicciones sobre el mejor modelo. Asigna el resultado a una variable con nombre `y_pred` (**Ejemplo: `y_pred = mejor_modelo.<<Función>>`**).

In [38]:
# your code here
y_pred = mejor_modelo.predict(x_test)

In [39]:
#---------- Celda de Pruebas ----------
# La variable "y_pred" existe
# La variable "y_pred" es un arreglo
# La variable "y_pred" tiene las dimensiones correctas
#--------------------------------------

# Se verifica que "y_pred" está definida
assert y_pred is not None, "Asegúrate de definir la variable con el nombre correcto."

# Se verifica que "y_pred" sea un arreglo
assert isinstance(y_pred, np.ndarray), "El resultado debe ser un arreglo de Numpy."

# Se evalúan las dimensiones de "y_pred"
assert y_pred.shape == (294,), "\'y_pred\' debe tener el mismo número de filas (predicciones) que \'x_test\' y \'y_test\'."
print("¡\'y_pred\' es un arreglo con las dimensiones correctas!")

¡'y_pred' es un arreglo con las dimensiones correctas!


### Ejercicio 4.3.

Utiliza el conjunto de predicciones `y_pred` y el conjunto de valores reales `y_test` para obtener la matriz de confusión del mejor modelo.

* Haz un llamado a la función que retorna la matriz de confusión como un arreglo. Asigna el resultado a una nueva variable con el nombre `p43` (**Ejemplo: `p43 = <<Función>>`**).
* Encontrarás la línea `p43` al final de la celda. Esta línea se usa para que puedas visualizar la matriz resultante, por lo que no la debes modificar.

In [40]:
# your code here

p43 = confusion_matrix(y_test, y_pred)

In [41]:
#---------- Celda de Pruebas ----------
# La variable "p43" existe
# La variable "p43" es un arreglo de numpy
# La variable "p43" tiene las dimensiones correctas
#--------------------------------------

# Se verifica que "p43" está definida
assert p43 is not None, "Asegúrate de definir la variable con el nombre correcto."

# Se verifica que "p43" sea un arreglo de numpy
assert isinstance(p43, np.ndarray), "El resultado debe ser un arreglo de Numpy."

# Se evalúan las dimensiones de "p43"
assert p43.shape == (2,2), "\'p43\' debe tener dos filas y dos columnas, equivalente al número de clases del problema."
print("¡\'p43\' es un arreglo con las dimensiones correctas!")

¡'p43' es un arreglo con las dimensiones correctas!


En total, puedes observar el rendimiento del mejor modelo usando `classification_report()`:

In [42]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91       245
           1       0.53      0.33      0.41        49

    accuracy                           0.84       294
   macro avg       0.70      0.63      0.66       294
weighted avg       0.82      0.84      0.82       294



## Cierre

Al realizar los ejercicios de este taller, has reforzado tus capacidades para construir pipelines y realizar una búsqueda exhaustiva de hiperparámetros, evaluando el mejor modelo utilizando la matriz de confusión y observando su reporte de clasificación.

---
*Creado por: Nicolás Díaz*  
*Revisado por: Haydemar Nuñez*  
*Versión de: Marzo 2, 2024*  
*Universidad de los Andes*   